In [ ]:

# coding: utf-8

# In[2]:

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import h5py
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
import numpy as np
import time
import torch.nn.init as ini
import multiprocessing
from multiprocessing import Queue
import random

class TwoPathConv(nn.Module):
    def __init__(self):
        super(TwoPathConv, self).__init__()
        self.local_conv1 = nn.Conv2d(2, 64, 7)
        self.local_conv2 = nn.Conv2d(64, 64, 3)
        self.local_conv3 = nn.Conv2d(2, 160, 13)
        self.total_conv = nn.Conv2d(224, 5, 21)

    def forward(self, x):
        under_x = F.relu(self.local_conv3(x))
        x = self.local_conv1(x)
        x = F.max_pool2d(F.relu(x), 4, stride = 1)
        x = self.local_conv2(x)
        x = F.max_pool2d(F.relu(x), 2, stride = 1)
        x = torch.cat((x, under_x), 1)
        x = self.total_conv(x)
        x = x.view(-1,5)
        return x

# In[3]:



In [ ]:
file_list = open('/home/yiqin/trainval-balanced.txt','r')
f = h5py.File('/home/yiqin/train.h5','r')
list1 = []
str1 = file_list.readlines()
for i in range(len(str1)):
    list1.append(str1[i][0:-1].split(' '))
print(len(list1))

import pickle as pkl
input1 = open('two_path_cnn/HG0001_Val_list.pkl', 'rb')
input2 = open('two_path_cnn/training_list.pkl', 'rb')
val_list = pkl.load(input1)
train_list = pkl.load(input2)
print(len(val_list))


#without multiprocessing
def create_sub_patch_phase1(size, key):
    training_patch = []
    training_label = []
    len_data = len(train_list)
    for i in range(size):
        case,x,y,z,l = train_list[key * size + i]
        x,y,z,l = int(x), int(y), int(z), int(l)
        case1 = case[0:2]
        case2 = case[3:]
        content = f[case1][case2]
        img_patch = content[0:2, x-16:x+16+1, y-16:y+16+1, z] #sample a 33x33 patch
        training_patch.append(img_patch)
        training_label.append(l)
    train_patch = torch.from_numpy(np.array(training_patch))
    train_label = torch.from_numpy(np.array(training_label))
    return train_patch, train_label


def create_val_patch(size):
    val_patch = []
    val_label = []
    len_data = len(val_list)
    for i in range(size*2, size*3):
        case,x,y,z,l = val_list[i]
        #print(i, key)
        x,y,z,l = int(x), int(y), int(z), int(l)
        case1 = case[0:2]
        case2 = case[3:]
        content = f[case1][case2]
        img_patch = content[0:2, x-16:x+16+1, y-16:y+16+1, z] #sample a 33x33 patch
        val_patch.append(img_patch)
        val_label.append(l)
    val_patch = torch.from_numpy(np.array(val_patch))
    val_label = torch.from_numpy(np.array(val_label))
    return val_patch, val_label


prev_time = time.clock()
num_epoch = 5
batch_size = 512
iteration = len(train_list) // batch_size
net = TwoPathConv()
#net.load_state_dict(torch.load('/home/yiqin/phase1_TwoPathConv_net.txt'))
net = net.cuda(0)

#net init
'''
for param in net.parameters():
    if len(param.size())==4:
        ini.uniform(param, a=-5e-3, b=5e-3)
'''        
#set hyperparams



In [ ]:
learning_rate = 5e-6
l1_reg = 5e-7
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum = 0.9, weight_decay = 5e-5)
scheduler = StepLR(optimizer, step_size=1, gamma=0.2)
#create val set
val_patch_size = 512
val_x, val_y = create_val_patch(val_patch_size)
val_x, val_y = Variable(val_x.cuda(0)), val_y.cuda(0)

for i in range(num_epoch):
    for j in range(iteration):
        training_patch, training_label = create_sub_patch_phase1(batch_size, j)
        x_train, y_train = Variable(training_patch.cuda(0)), Variable(training_label.cuda(0), requires_grad=False)
        #train
        y_pred = net.forward(x_train)
        y_pred = y_pred.view(-1,5)
        loss = F.cross_entropy(y_pred, y_train)#cross entropy loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #check accuracy
        if j%10 == 0:
            print('iteration %d /%d:'%(j, iteration), loss.data)
            print(float(j)/(iteration * num_epoch),  'finished')
            val_pred = net.forward(val_x)
            val_pred = val_pred.view(-1, 5)
            _, predicted = torch.max(val_pred.data, 1)
            correct = (predicted == val_y).sum()
            print('Validation accuracy:', float(correct) / val_patch_size)
            print('time used:%.3f'% (time.clock() - prev_time))
    scheduler.step()
    output = "/home/yiqin/two_path_cnn/p1_TPWconv_net" + str(i) + '.txt'
    torch.save(net.state_dict(), output)
    
print ("phase1: successfully trained!")

print ("phase1: successfully saved!")


In [ ]:
import random
def create_patch(f, file_list, patch_size = 100):
    training_patch = []
    training_label = []
    for i in range(patch_size):
        index = random.randint(0, len(file_list))
        case,x,y,z,l = file_list[index]
        x,y,z,l = int(x), int(y), int(z), int(l)
        case1 = case[0:2]
        case2 = case[3:]
        img_patch = f[case1][case2][:, x-16:x+16+1, y-16:y+16+1, z] #sample a 33x33 patch
        training_patch.append(img_patch)
        training_label.append(l)
    training_patch = torch.from_numpy(np.array(training_patch))
    training_label = torch.from_numpy(np.array(training_label))
    return training_patch, training_label

def create_val_patch(f, file_list, step = 10000, key = 9999):
    val_patch = []
    val_label = []
    patch_size = len(file_list) // step
    for i in range(patch_size):
        case,x,y,z,l = file_list[i * step + key]
        x,y,z,l = int(x), int(y), int(z), int(l)
        case1 = case[0:2]
        case2 = case[3:]
        img_patch = f[case1][case2][:, x-16:x+16+1, y-16:y+16+1, z] #sample a 33x33 patch
        val_patch.append(img_patch)
        val_label.append(l)
    val_patch = torch.from_numpy(np.array(val_patch))
    val_label = torch.from_numpy(np.array(val_label))
    return val_patch, val_label
# subsampling
def create_sub_patch(f, file_list, step = 100, key = 0):
    training_patch = []
    training_label = []
    patch_size = len(file_list) // step
    for i in range(patch_size):
        case,x,y,z,l = file_list[i * step + key]
        #print(i, key)
        x,y,z,l = int(x), int(y), int(z), int(l)
        case1 = case[0:2]
        case2 = case[3:]
        img_patch = f[case1][case2][:, x-16:x+16+1, y-16:y+16+1, z] #sample a 33x33 patch
        training_patch.append(img_patch)
        training_label.append(l)
    training_patch = torch.from_numpy(np.array(training_patch))
    training_label = torch.from_numpy(np.array(training_label))
    return training_patch, training_label

In [ ]:
#1 phase train
import h5py
file_list = open('trainval-balanced.txt','r')
f = h5py.File('training.h5','r')
list1 = []
str1 = file_list.readlines()
for i in range(len(str1)):
    list1.append(str1[i][0:-1].split(' '))


In [ ]:
#train & validation
import numpy as np
import time
net = torch.load('prematrue_net.pkl')
learning_rate = 1e-4
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

epoch_start = 1000
epoch_end = 2000
train_patch_size = len(list1) //  20000
val_patch_size = len(list1) //  20000
net.cuda(1)
prev_time = time.clock()
for i in range(epoch_start, epoch_end):
    training_patch, training_label = create_sub_patch(f = f, file_list = list1, step = 20000, key = i)
    x_train, y_train = Variable(training_patch.cuda(1)), Variable(training_label.cuda(1), requires_grad=False)
    #train
    optimizer.zero_grad()
    y_pred = net.forward(x_train)
    y_pred = y_pred.view(train_patch_size,5)
    loss = F.cross_entropy(y_pred, y_train)
    loss.backward()
    print(loss)
    optimizer.step()
    #check accuracy
    val_patch, val_y = create_val_patch(f = f, file_list = list1, step = 20000, key = 5000)
    val_y = val_y.view((val_patch_size,1))
    val_x = Variable(val_patch.cuda(1))
    val_pred = net.forward(val_x)
    val_pred = val_pred.view(val_patch_size, 5)
    _, predicted = torch.max(val_pred.data, 1)
    #print(type(predicted))
    total = val_patch_size
    correct = (predicted == val_y.cuda(1)).sum()
    print 'epoch: %d'%i 
    print'Validation accuracy:', float(correct) / total
    print'time used:%.3f'% (time.clock() - prev_time)
torch.save(net, 'premature_net.pkl')

In [ ]:
#2nd phase train
import h5py
normal_list = open('trainval.txt','r')
f = h5py.File('training.h5','r')
list2 = []
str2 = normal_list.readlines()
for i in range(len(str2)):
    list2.append(str2[i][0:-1].split(' '))
print(len(list2))

In [ ]:
#check accuracy
import numpy as np
import time
net = TwoPathConv()
net.load_state_dict(torch.load("trained_model.txt"))
learning_rate = 1e-4
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

val_patch_size = len(list2) //  20000
num_epoch = 1
train_patch_size = 500
net.cuda(1)
prev_time = time.clock()
for i in range(num_epoch):
    #check accuracy
    val_patch, val_y = create_val_patch(f = f, file_list = list2, step = 20000)
    val_y = val_y.view((val_patch_size,1))
    val_x = Variable(val_patch.cuda(1))
    val_pred = net.forward(val_x)
    #print(val_pred)
    #print('-------------')
    #print(val_y)
    val_pred = val_pred.view(val_patch_size, 5)
    _, predicted = torch.max(val_pred.data, 1)
    #print(type(predicted))
    total = float(val_patch_size)
    correct = (predicted == val_y.cuda(1)).sum()
    print'Validation accuracy:', float(correct) / total
    print'time used:%.3f'% (time.clock() - prev_time)
    


In [ ]:
f.close()

In [ ]:
import h5py
import numpy as np
f = h5py.File('/Users/Fischer/Tumor/TumorSeg/data/training.h5','r')
list1 = []
str1 = file_list.readlines()
for i in range(len(str1)):
    list1.append(str1[i][0:-1].split(' '))
print(len(list1))
training_patch = []
training_label = []
for i in range(100):
    case,x,y,z,l = list1[i]
    x,y,z,l = int(x), int(y), int(z), int(l)
    case1 = case[0:2]
    case2 = case[3:]
    img_patch = f[case1][case2][:, x-16:x+16+1, y-16:y+16+1, z] #sample a 33x33 patch
    training_patch.append(img_patch)
    training_label.append(l)
training_patch = np.array(training_patch)
training_label = np.array(training_label)

print(training_patch[0])
print(training_label[0])

In [ ]:
from medpy.io import load
img = load('VSD.Brain.XX.O.MR_Flair.865.mha')
print(img)
#plt.imshow(array_image2[30,:,:], cmap = 'gray')

In [ ]:
import SimpleITK as sitk
import matplotlib.pyplot as plt
input_image = sitk.ReadImage('VSD.Brain.XX.O.MR_Flair.17572.mha')
image_2D = sitk.Image(64, 64, sitk.sitkFloat32)
print(input_image.GetSize())
#help(image_2D)
array_image = sitk.GetArrayFromImage(input_image)
print(array_image[50,50,:])
variable_image = Variable(torch.from_numpy(array_image))
#print(variable_image)

plt.imshow(array_image[30,:,:], cmap = 'gray')

In [ ]:
#!/usr/bin/env python

r"""Convert nii to hdf5 file.

Example usage:
    ./create_h5.py --data_dir=/path/to/data_dir_contain_HG_and_LG \
        --output_path=/path/to/h5_file
"""
import os.path as osp
import argparse

import numpy as np
import h5py
import nibabel as nib

# SAMPLE and MODALITY
SAMPLE = ["LG/0001", "LG/0002", "LG/0004", "LG/0006", "LG/0008", "LG/0011",
          "LG/0012", "LG/0013", "LG/0014", "LG/0015", "HG/0001", "HG/0002",
          "HG/0003", "HG/0004", "HG/0005", "HG/0006", "HG/0007", "HG/0008",
          "HG/0009", "HG/0010", "HG/0011", "HG/0012", "HG/0013", "HG/0014",
          "HG/0015", "HG/0022", "HG/0024", "HG/0025", "HG/0026", "HG/0027"]
MODALITY = ["Flair.finalNorm.nii.gz", "T1.finalNorm.nii.gz",
            "T1c.finalNorm.nii.gz", "T2.finalNorm.nii.gz"]

def parse_args():
    """ Parse args.
    """
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir', dest='input_dir',
                        help='dir for training nii data',
                        default='training', type=str)
    parser.add_argument('--output_path', dest='output_path',
                        help='output path for h5 file',
                        default='training.h5', type=str)
    args = parser.parse_args()
    return args

def get_img_data(data_dir, sample, modality):
    """Get image data.
    """
    nii_path = osp.join(data_dir, sample, modality)
    img = nib.load(nii_path)
    return img.get_data()

def main():
    """main loop
    """
    args = parse_args()
    data_dir = args.input_dir
    with h5py.File(args.output_path, 'w') as f:
        for path in SAMPLE:
            img_data = get_img_data(data_dir, path, MODALITY[0])
            dim = img_data.shape
            img_array = np.empty((4, dim[0], dim[1], dim[2]), dtype=np.float32)
            img_array[0, ...] = img_data
            for j in range(1, len(MODALITY)-1):
                img_array[j, ...] = get_img_data(data_dir, path, MODALITY[j])
            f[path] = img_array

if __name__ == '__main__':
    main()
